In [2]:
from BVDDataAnalyzer import outputSpec as outputSpec
import BVDDataAnalyzer
import sys
import csvTools
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [3]:
analyzer = BVDDataAnalyzer.Analyzer.SimulationAnalyzer('/net/m07/nwg/SmallTests/testing_list.txt')

numberOfFarms, allIneed = analyzer.file.getEndemicData()
path = analyzer.file.path

times = sorted(allIneed.keys())

In [4]:
csvFarmDataDict = {}
tonetime = times[0]
for key in allIneed[tonetime]["prevalenceCows"]:
    csvFarmDataDict[key] = []
csvFarmDataDict["numberOfInfectedFarms"] = []
for time in times:
    thisData = allIneed[time]
    
    for key in thisData["prevalenceCows"]:
        csvFarmDataDict[key].append(thisData["prevalenceCows"][key])

    csvFarmDataDict["numberOfInfectedFarms"].append(len(thisData["prevalenceFarms"]))

csvTools.writeDictOfLists(path+"/farm_data.csv",csvFarmDataDict)

In [24]:
name = ['pi', '# inf. Farms', 't', 's', 'r', 'ti', 'Total']
df = pd.read_csv('/net/m07/nwg/SmallTests/farm_data.csv', sep = ',', header = 0, names = name)

In [25]:
df.head()

,pi,# inf. Farms,t,s,r,ti,Total
0,0,1079,0,231766,88122,6551,326439
1,0,1079,5,231766,88122,6551,326439
2,0,1079,10,231766,88122,6551,326439
3,0,1079,15,231766,88122,6551,326439
4,0,1079,20,231766,88122,6551,326439


In [43]:
dfbuf = df.drop({'s', 'r', 'ti', 'Total'},1)
dfbuf.head()

,pi,# inf. Farms,t
0,0,1079,0
1,0,1079,5
2,0,1079,10
3,0,1079,15
4,0,1079,20


In [44]:
dfbuf = dfbuf.reindex_axis(['t', '# inf. Farms', 'pi'], axis=1)
dfbuf.head()

,t,# inf. Farms,pi
0,0,1079,0
1,5,1079,0
2,10,1079,0
3,15,1079,0
4,20,1079,0


In [45]:
starting_date = '2008-01-01'
begin = datetime.strptime(starting_date, '%Y-%m-%d')
dfbuf['DATE'] = dfbuf['t'].map( lambda x : timedelta(x)+begin)
dfbuf["Month"] = dfbuf.DATE.map( lambda x : x.month)
dfbuf["Year"] = dfbuf.DATE.map( lambda x : x.year)
#dfbuf.drop({"DATE", "t"}, axis=1, inplace=True)
dfbuf.head()

,t,# inf. Farms,pi,DATE,Month,Year
0,0,1079,0,2008-01-01,1,2008
1,5,1079,0,2008-01-06,1,2008
2,10,1079,0,2008-01-11,1,2008
3,15,1079,0,2008-01-16,1,2008
4,20,1079,0,2008-01-21,1,2008


In [57]:
dfbuf2 = dfbuf.groupby(['Month',"Year"]).aggregate({"pi": sum, "# inf. Farms":sum,"DATE": min, "t":min})
dfbuf2.reset_index(inplace=True)
dfbuf2.sort_values("DATE", inplace=True)
dfbuf2.index = np.arange(len(dfbuf2))
dfbuf2.head(20)

,Month,Year,DATE,pi,t,# inf. Farms
0,1,2008,2008-01-01,0,0,7553
1,2,2008,2008-02-05,0,35,5395
2,3,2008,2008-03-01,0,60,7553
3,4,2008,2008-04-05,0,95,6474
4,5,2008,2008-05-05,0,125,6474
5,6,2008,2008-06-04,0,155,6474
6,7,2008,2008-07-04,0,185,6474
7,8,2008,2008-08-03,0,215,6474
8,9,2008,2008-09-02,0,245,6474
9,10,2008,2008-10-02,10648,275,6280


In [ ]:
dfbuf2.aggregate({"pi": sum, "# inf. Farms":sum})